In [ ]:
import pandas as pd
import numpy as np
import re
import textwrap
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from scipy.spatial import distance
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install sumy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 65.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21693 sha256=07fd371879e12deb4be5b0447a41a8730b27e0e441a9f3dd18ac96580e2c8943
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=611f2f8b4a60264afa3a88cda952a2acf71618a24ddc7ceb62e7f9e479188716
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=1068

In [ ]:
with open('/content/drive/My Drive/bbc_text_cls.csv', 'r') as f:
    content = f.read()
print(content)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
import pandas as pd

csv_path = '/content/drive/My Drive/bbc_text_cls.csv'
df = pd.read_csv(csv_path)

print(df)

                                                   text    labels
0     Ad sales boost Time Warner profit\n\nQuarterly...  business
1     Dollar gains on Greenspan speech\n\nThe dollar...  business
2     Yukos unit buyer faces loan claim\n\nThe owner...  business
3     High fuel prices hit BA's profits\n\nBritish A...  business
4     Pernod takeover talk lifts Domecq\n\nShares in...  business
...                                                 ...       ...
2220  BT program to beat dialler scams\n\nBT is intr...      tech
2221  Spam e-mails tempt net shoppers\n\nComputer us...      tech
2222  Be careful how you code\n\nA new European dire...      tech
2223  US cyber security chief resigns\n\nThe man mak...      tech
2224  Losing yourself in online gaming\n\nOnline rol...      tech

[2225 rows x 2 columns]


In [ ]:
doc = df[df.labels == 'business']['text'].sample(random_state=24)


In [ ]:
print(doc)

370    Singapore growth at 8.1% in 2004\n\nSingapore'...
Name: text, dtype: object


In [ ]:
import textwrap

def wrap(x):
    return textwrap.fill(x, replace_whitespace=False, fix_sentence_endings=True)

input_text = "This is a long piece of text that needs to be wrapped properly in a formatted way. It should consider sentence endings and not replace existing whitespace."

wrapped_text = wrap(input_text)
print(wrapped_text)


This is a long piece of text that needs to be wrapped properly in a
formatted way.  It should consider sentence endings and not replace
existing whitespace.


In [ ]:
def wrap(x):
  return textwrap.fill(x, replace_whitespace=False, fix_sentence_endings=True)



In [ ]:
print(wrap(doc.iloc[0]))


Singapore growth at 8.1% in 2004

Singapore's economy grew by 8.1% in
2004, its best performance since 2000, figures from the trade ministry
show.

The advance, the second-fastest in Asia after China, was led by
growth of 13.1% in the key manufacturing sector.  However, a slower-
than-expected fourth quarter points to more modest growth for the
trade-driven economy in 2005 as global technology demand falls back.
Slowdowns in the US and China could hit electronics exports, while the
tsunami disaster may effect the service sector.

Economic growth is
set to halve in Singapore this year to between 3% and 5%. In the
fourth quarter, the city state's gross domestic product (GDP) rose at
an annual rate of 2.4%. That was up from the third quarter, when it
fell 3.0%, but was well below analyst forecasts.  "I am surprised at
the weak fourth quarter number.  The main drag came from electronics,"
said Lian Chia Liang, economist at JP Morgan Chase.  Singapore's
economy had contracted over the summe

In [ ]:
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
summarizer = TextRankSummarizer()
parser = PlaintextParser.from_string(
    doc.iloc[0].split("\n", 1)[1],
    Tokenizer("russian"))
summary = summarizer(parser.document, sentences_count=3)
sum_=str()
for s in summary:
  sum_+=str(s)
sum_

"The advance, the second-fastest in Asia after China, was led by growth of 13.1% in the key manufacturing sector.Slowdowns in the US and China could hit electronics exports, while the tsunami disaster may effect the service sector.The economy's poor performance in the July to September period followed four consecutive quarters of double-digit growth as Singapore bounced back strongly from the effects of the deadly Sars virus in 2003."

In [ ]:
featurizer = TfidfVectorizer(stop_words=stopwords.words('english'),norm='l1')
# mean of tfidf score for each sencence
def get_sentence_score(tfidf_row):
  x = tfidf_row[tfidf_row != 0]
  return x.mean()

def summarize_tfidf(text):
  # tokenize sentences
  sents = nltk.sent_tokenize(text)
  # tf-idf score
  X = featurizer.fit_transform(sents)
  # scores for each sentence
  scores = np.zeros(len(sents))
  for i in range(len(sents)):
    score = get_sentence_score(X[i,:])
    scores[i] = score
  # sort the scores and pick top 5 sentences
  sort_idx = np.argsort(-scores)
  sort_idx=sort_idx[:5]
  sort_idx=np.sort(sort_idx)
  # generate summary
  sum_=str()
  for i in sort_idx[:5]:
    sum_+=sents[i]
  return sum_
  sum_
  print(sum_)

In [ ]:
print(sum_)

The advance, the second-fastest in Asia after China, was led by growth of 13.1% in the key manufacturing sector.Slowdowns in the US and China could hit electronics exports, while the tsunami disaster may effect the service sector.The economy's poor performance in the July to September period followed four consecutive quarters of double-digit growth as Singapore bounced back strongly from the effects of the deadly Sars virus in 2003.


In [ ]:
sum_tfidf=summarize_tfidf(doc.iloc[0].split("\n", 1)[1])


In [ ]:
print(wrap(sum_tfidf))


The advance, the second-fastest in Asia after China, was led by growth
of 13.1% in the key manufacturing sector.Economic growth is set to
halve in Singapore this year to between 3% and 5%.That was up from the
third quarter, when it fell 3.0%, but was well below analyst
forecasts."I am surprised at the weak fourth quarter
number.Singapore's economy had contracted over the summer, weighed
down by soaring oil prices.


COSINE SIMILARITY STARTS SUMMARIZATION ALGO-2


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def summarize_cosine(text, factor = 0.15):
  # tokenize sentences
  sents = nltk.sent_tokenize(text)
  # calulate tf-idf score
  featurizer = TfidfVectorizer(stop_words=stopwords.words('english'),norm='l1')
  X = featurizer.fit_transform(sents)
  # compute similarity matrix
  S = cosine_similarity(X)
  # normalize similarity matrix
  S /= S.sum(axis=1, keepdims=True)
  # uniform transition matrix
  U = np.ones_like(S) / len(S)
  # smoothed similarity matrix
  S = (1 - factor) * S + factor * U
  # find the limiting / stationary distribution
  eigenvals, eigenvecs = np.linalg.eig(S.T)
  # compute scores
  scores = eigenvecs[:,0] / eigenvecs[:,0].sum()
  # sort the scores and pick top 5 sentences
  sort_idx = np.argsort(-scores)
  sort_idx=sort_idx[:5]
  sort_idx=np.sort(sort_idx)
  # generate summary
  sum_=str()
  for i in sort_idx[:5]:
    #print(wrap("%.2f: %s" % (scores[i], sents[i])))
    sum_+=sents[i]
  return sum_

In [ ]:
sum_cos=summarize_cosine(doc.iloc[0].split("\n", 1)[1], factor = 0.15)




In [ ]:
print(wrap(sum_cos))



Singapore's economy grew by 8.1% in 2004, its best performance since
2000, figures from the trade ministry show.However, a slower-than-
expected fourth quarter points to more modest growth for the trade-
driven economy in 2005 as global technology demand falls back.Economic
growth is set to halve in Singapore this year to between 3% and 5%."I
am surprised at the weak fourth quarter number.The economy's poor
performance in the July to September period followed four consecutive
quarters of double-digit growth as Singapore bounced back strongly
from the effects of the deadly Sars virus in 2003.


K MEANS CLUSTERING everything about

In [ ]:
sentence = nltk.sent_tokenize(doc.iloc[0].split("\n", 1)[1])
corpus = []
for i in range(len(sentence)):
    sen = re.sub('[^a-zA-Z]', " ", sentence[i])
    sen = sen.lower()
    sen = sen.split()
    sen = ' '.join([i for i in sen if i not in stopwords.words('english')])
    corpus.append(sen)

all_words = [i.split() for i in corpus]
model = Word2Vec(all_words, min_count=1,vector_size=300)

sent_vector=[]
for i in corpus:
    plus=0
    for j in i.split():
        plus+= model.wv[j]
    plus = plus/len(i.split())
    sent_vector.append(plus)

n_clusters = 5
kmeans = KMeans(n_clusters, init = 'k-means++', random_state = 42)
y_kmeans = kmeans.fit_predict(sent_vector)

my_list=[]
for i in range(n_clusters):
    my_dict={}
    for j in range(len(y_kmeans)):
        if y_kmeans[j]==i:
            my_dict[j] =  distance.euclidean(kmeans.cluster_centers_[i],sent_vector[j])
    min_distance = min(my_dict.values())
    my_list.append(min(my_dict, key=my_dict.get))
sum_kmean=str()
for i in sorted(my_list):
    sum_kmean+=sentence[i]

print(wrap(sum_kmean))

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(



Singapore's economy grew by 8.1% in 2004, its best performance since
2000, figures from the trade ministry show.Economic growth is set to
halve in Singapore this year to between 3% and 5%.That was up from the
third quarter, when it fell 3.0%, but was well below analyst
forecasts."I am surprised at the weak fourth quarter number.The
economy's poor performance in the July to September period followed
four consecutive quarters of double-digit growth as Singapore bounced
back strongly from the effects of the deadly Sars virus in 2003.


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
t1=word_tokenize(sum_)
t2=word_tokenize(sum_tfidf)
t3=word_tokenize(sum_cos)
t4=word_tokenize(sum_kmean)
print("Bleu score of tfidf model: ",sentence_bleu(t1,t2,weights=(1,0,0,0)))
print("Bleu score of cosine similarity model: ",sentence_bleu(t1,t3,weights=(1,0,0,0)))
print("Bleu score of kmeans clustering model: ",sentence_bleu(t1,t4,weights=(1,0,0,0)))


Bleu score of tfidf model:  0.047619047619047616
Bleu score of cosine similarity model:  0.04807692307692307
Bleu score of kmeans clustering model:  0.03921568627450981


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
import PyPDF2
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from scipy.spatial import distance
import textwrap
from nltk.translate.bleu_score import sentence_bleu

# Load NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# PDF Parsing
def parse_pdf(pdf_path):
    text_by_page = []
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page in pdf_reader.pages:
            text_by_page.append(page.extract_text())
    return text_by_page

# Your Summarization Algorithm
def summarize_page(text):
    # Your summarization algorithm here
    # ... (include the code for your summarization algorithm)

# Main Process
def main(pdf_path):
    text_by_page = parse_pdf(pdf_path)
    reference_summaries = [
        "Reference summary for page 1.",
        "Reference summary for page 2.",
        "Reference summary for page 3.",
        # Add more reference summaries for each page
    ]
    for page_num, (page_text, reference_summary) in enumerate(zip(text_by_page, reference_summaries)):
        generated_summary = summarize_page(page_text)
        bleu_score = sentence_bleu([reference_summary], generated_summary)
        print(f"Page {page_num + 1} BLEU Score: {bleu_score:.4f}")
        print("Generated Summary:", generated_summary)
        print("Reference Summary:", reference_summary)
        print("-" * 30)

# Replace with your PDF file path
pdf_file_path = 'path_to_your_pdf.pdf'
main(pdf_file_path)


IndentationError: ignored

In [ ]:
!pip install PyPDF2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.3 MB/s eta 0:00:00


In [ ]:
import PyPDF2


breaking pdf and calculating scores


In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from scipy.spatial import distance

# Load NLTK resources
nltk.download('punkt')
nltk.download('stopwords')


def parse_pdf(pdf_path):
    text_by_page = []
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page in pdf_reader.pages:
            text_by_page.append(page.extract_text())
    return text_by_page


def summarize_page(text):
    # Preprocess the input text
    sentence = nltk.sent_tokenize(text)
    corpus = []
    for i in range(len(sentence)):
        sen = re.sub('[^a-zA-Z]', " ", sentence[i])
        sen = sen.lower()
        sen = sen.split()
        sen = ' '.join([i for i in sen if i not in stopwords.words('english')])
        corpus.append(sen)

    # Train Word2Vec model
    all_words = [i.split() for i in corpus]
    model = Word2Vec(all_words, min_count=1, vector_size=300)

    # Generate sentence vectors using Word2Vec
    sent_vector = []
    for i in corpus:
        plus = 0
        for j in i.split():
            plus += model.wv[j]
        plus = plus / len(i.split())
        sent_vector.append(plus)

    # Apply K-Means clustering
    n_clusters = 5
    kmeans = KMeans(n_clusters, init='k-means++', random_state=42)
    y_kmeans = kmeans.fit_predict(sent_vector)

    my_list = []
    for i in range(n_clusters):
        my_dict = {}
        for j in range(len(y_kmeans)):
            if y_kmeans[j] == i:
                my_dict[j] = distance.euclidean(kmeans.cluster_centers_[i], sent_vector[j])
        min_distance = min(my_dict.values())
        my_list.append(min(my_dict, key=my_dict.get))

    # Generate the final summary using selected sentences
    generated_summary = ""
    for i in sorted(my_list):
        generated_summary += sentence[i]

    return generated_summary

# Main Process
def main(pdf_path):
    text_by_page = parse_pdf(pdf_path)
    reference_summaries = ["Seldom does saturn and Morae,juxtapose and mate in the great fabric of Gaea, as it did to rekindle my thread of life with her;## the writer here wanted to stress the great luck and the timing of it, involved on this planet, that ## morae -goddess of fate , saturn god of time,gaea-goddess of earthunderlined , his rekindling contact with the protagonist.",
"In this cradle of desolation reeking of grim profound, she bounced around,emanating a pervading tang of joy,boundless and intense;## in this world of endless despair and grim, the protagonist had somehow found a way to thrive and bounced around sharing and inducing this infective spirit on all the souls she touched, and influenced. Her presence created an endless joy which was anything but temporary as she changed the very worldview of people she touched.",
        "##lying underneath her visible grace and poise duly complemented with her calm and joy Lay a scarred and equally humane existence(represented by a ship wreck) with her share of loss , pain and regrets, which she had buried deep down her bubbly self."


        # Add more reference summaries for each page
    ]
    for page_num, (page_text, reference_summary) in enumerate(zip(text_by_page, reference_summaries)):
        generated_summary = summarize_page(page_text)
        bleu_score = sentence_bleu([reference_summary], generated_summary)
        print(f"Page {page_num + 1} BLEU Score: {bleu_score:.4f}")
        print("Generated Summary:", generated_summary)
        print("Reference Summary:", reference_summary)
        print("-" * 30)

# Replace with your PDF file path
pdf_file_path = '/content/drive/MyDrive/POEM_EXPLANATION.pdf'
main(pdf_file_path)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Page 1 BLEU Score: 0.0179
Generated Summary: SHE
:
A
POR TRAIT
S e l d o m
d o e s
s a t u r n
a n d
M o r a e
, j u x t a p o s e
a n d
m a t e
i n
t h e
g r e a t
f a b r i c
o f
G a e a
,
a s
i t
d i d
t o
r e k i n d l e
m y
t h r e a d
o f
l i f e
w i t h
h e r ;
# #
t h e
w r i t e r
h e r e
w a n t e d
t o
s t r e s s
t h e
g r e a t
l u c k
a n d
t h e
t i m i n g
o f
i t ,
i n v o l v e d
o n
t h i s
p l a n e t ,
t h a t
# #
m o r a e
- g o d d e s s
o f
f a t e
,
s a t u r n
g o d
o f
t i m e , g a e a - g o d d e s s
o f
e a r t h
u n d e r l i n e d
,
h i s
r e k i n d l i n g
c o n t a c t
w i t h
t h e
p r o t a g o n i s t .F o r
m y
d a m p
, p a l e
c o a l s
o f
e x u b e r a n c e
a n d
h a l e , b u r s t
i n t o
f l a m e s
o f
b o n h o m i e , p r e v i o u s l y
u n c h a r t e d , u p o n
h e r
a b r u p t , t e n d e r
e m b r a c e
o f
m y
a r i d
l a n d s c a p e
o f
m i n e ,
m e a g e r , s i c k l y
y e t
a s p i r a t i o n a l
e x i s t e n c e
o n
t 

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Page 2 BLEU Score: 0.0154
Generated Summary: I n
t h i s
c r a d l e
o f
d e s o l a t i o n
r e e k i n g
o f
g r i m
p r o f o u n d ,
s h e
b o u n c e d
a r o u n d , e m a n a t i n g
a
p e r v a d i n g
t a n g
o f
j o y , b o u n d l e s s
a n d
i n t e n s e ;
# #
i n
t h i s
w o r l d
o f
e n d l e s s
d e s p a i r
a n d
g r i m ,
t h e
p r o t a g o n i s t
h a d
s o m e h o w
f o u n d
a
w a y
t o
t h r i v e
a n d
b o u n c e d
a r o u n d
s h a r i n g
a n d
i n d u c i n g
t h i s
i n f e c t i v e
s p i r i t
o n
a l l
t h e
s o u l s
s h e
t o u c h e d ,
a n d
i n f l u e n c e d .H e r
p r e s e n c e
c r e a t e d
a n
e n d l e s s
j o y
w h i c h
w a s
a n y t h i n g
b u t
t e m p o r a r y
a s
s h e
c h a n g e d
t h e
v e r y
w o r l d v i e w
o f
p e o p l e
s h e
t o u c h e d
S h e
s w i r l e d , d a n c e d
i n
t h i s
l i f e l e s s , s a n g u i n e
c e m e t e r y
o f
h o p e s
a n d
q u a s h e d
a s p i r a t i o n s ,
r e d e e m i n g
a n d
e n r i 

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
